In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, RobustScaler, MinMaxScaler

import pandas as pd
import numpy as np

from scripts import load_data
from scripts import machine_learning

In [2]:
df = load_data.load_df()
df_transformed = machine_learning.transform_data(df)
                    

X, y = machine_learning.split_X_y(df_transformed)
tts = train_test_split(X, y, stratify=y, test_size=0.3)
X_train, X_test, y_train, y_test = tts

In [3]:
model_table = machine_learning.find_random_forest_class_parameters(X[['home_club_position', 'away_club_position', 'attendance',
       'squad_size_x', 'average_age_x', 'foreigners_percentage_x',
       'national_team_players_x']], y, cv=5)

KeyboardInterrupt: 

In [5]:
model_grid = pd.DataFrame(model_table)

In [6]:
model_grid.head(2)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gaussiannb__priors,param_gaussiannb__var_smoothing,param_simpleimputer__strategy,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.030564,0.002787,0.005223,0.000755,None,0.0,mean,"{'gaussiannb__priors': None, 'gaussiannb__var_...",0.531476,0.516701,0.516663,0.522364,0.522444,0.521930,0.005416,2
1,0.052561,0.001306,0.004867,0.000641,None,0.0,median,"{'gaussiannb__priors': None, 'gaussiannb__var_...",0.530111,0.514855,0.515378,0.520758,0.520116,0.520243,0.005484,4


In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder

def transform_data(df):
    df_copy = df.copy()

    cat_transformer = make_pipeline(
        OrdinalEncoder(categories="auto"),
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("result", cat_transformer, ["result"]),
        ]
    )

    transformed_values = preprocessor.fit_transform(df_copy)

    categories_result = preprocessor.named_transformers_["result"] \
        .named_steps["ordinalencoder"].categories_[0]

   

    return categories_result




In [7]:
model_grid.loc[model_grid['rank_test_score']==1]['params'].values

array([{'gaussiannb__priors': None, 'gaussiannb__var_smoothing': 1e-06, 'simpleimputer__strategy': 'mean'}],
      dtype=object)

In [41]:
from sklearn.metrics import accuracy_score

params = {'gaussiannb__priors': None, 
          'gaussiannb__var_smoothing': 1e-06, 
          'simpleimputer__strategy': 'mean'}

model = machine_learning.predict_gaussian_model(params=params)
model.fit(X[['home_club_position', 'away_club_position', 'attendance',
       'squad_size_x', 'average_age_x', 'foreigners_percentage_x',
       'national_team_players_x']], y)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Dokładność modelu:", accuracy)




Dokładność modelu: 0.5230448048819656


ValueError: Found input variables with inconsistent numbers of samples: [62267, 18681]

In [ ]:
    "home_club_position",
            "away_club_position",
            "attendance",
            "squad_size_x",
            "average_age_x",
            "foreigners_percentage_x",
            "national_team_players_x",
            "result",

In [13]:
models_list = {machine_learning.test_linear_model: machine_learning.create_linear_model,
               machine_learning.test_gausian_model: machine_learning.create_gausian_model}
               
for model_test in models_list.items():
    params = model_test[0](X[["home_club_position","away_club_position",]]
                                 , y, cv = 2)
    
    model = model_test[1](params=params)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(model, score)
    


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('polynomialfeatures', PolynomialFeatures(degree=4)),
                ('linearregression', LinearRegression(n_jobs=1))]) 0.10024192059917658
Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('gaussiannb', GaussianNB())]) 0.5211712435094481


In [20]:
model = RandomForestClassifier(n_estimators=100, max_depth=10)
RandomForestClassifier(
    n_estimators=100,
    criterion='gini', max_depth=None, min_samples_split=2,
    min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt',
    max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True,
    oob_score=False, n_jobs=None, random_state=None, verbose=0,
    warm_start=False, class_weight=None, ccp_alpha=0.0,
    max_samples=None)

RandomForestClassifier()

In [21]:
m.fit(X_train, y_train)
m.score(X_test, y_test)

0.10770078251971538

In [7]:
model.score(X_test, y_test)

0.5615866388308977

In [22]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)
model.score(X_test, y_test)

ValueError: Input X contains NaN.
GaussianNB does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [15]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.5182270756383491

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred, normalize="all")

array([[0.02772871, 0.09988759, 0.08837857],
       [0.02660457, 0.23157219, 0.07280124],
       [0.03725711, 0.15684385, 0.25892618]])

In [18]:
import seaborn as sns
import pandas as pd
# create confusion matrix dataframe with labels
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=["win", "loss", 'draw'], columns=["win", "loss", 'draw'])
df_cm

,win,loss,draw
win,518,1866,1651
loss,497,4326,1360
draw,696,2930,4837


In [10]:
from sklearn.ensemble import VotingClassifier
model1 = LogisticRegression()
model2 = DecisionTreeClassifier(max_depth=2)
model3 = GaussianNB()
model = VotingClassifier([("lr", model1), ("dt", model2), ("nb", model3)])
model.fit(X_train, y_train)
model.score(X_test, y_test)

NameError: name 'GaussianNB' is not defined